# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
# Libraries

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [32]:
import pandas as pd

file_path = r'C:\\Users\\David Santos\\Desktop\\Labs IronHack\\Semi-Prontos\\7 - lab-advanced-pandas\\your-code\\data\\Admission_Predict.csv'
db = pd.read_csv(file_path)

Remove trailing spaces at the end of the column names if there are any.


In [33]:
db.columns  = db.columns.str.strip()

db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Let's evaluate the dataset by looking at the `head` function.

In [34]:
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [35]:
missing_data = db.isnull().sum() 

print (missing_data)

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [36]:
db = db.set_index("Serial No.")
print(db) 

            GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  \
Serial No.                                                              
1                 337          118                  4  4.5  4.5  9.65   
2                 316          104                  3  3.0  3.5  8.00   
3                 322          110                  3  3.5  2.5  8.67   
4                 314          103                  2  2.0  3.0  8.21   
5                 330          115                  5  4.5  3.0  9.34   
...               ...          ...                ...  ...  ...   ...   
381               324          110                  3  3.5  3.5  9.04   
382               325          107                  3  3.0  3.5  9.11   
383               330          116                  4  5.0  4.5  9.45   
384               312          103                  3  3.5  4.0  8.78   
385               333          117                  4  5.0  4.0  9.66   

            Research  Chance of Admit  
Serial No.

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [37]:
db_with_index = db.set_index(["GRE Score", "CGPA"])
print(db_with_index)

                TOEFL Score  University Rating  SOP  LOR  Research  \
GRE Score CGPA                                                       
337       9.65          118                  4  4.5  4.5         1   
316       8.00          104                  3  3.0  3.5         1   
322       8.67          110                  3  3.5  2.5         1   
314       8.21          103                  2  2.0  3.0         0   
330       9.34          115                  5  4.5  3.0         1   
...                     ...                ...  ...  ...       ...   
324       9.04          110                  3  3.5  3.5         1   
325       9.11          107                  3  3.0  3.5         1   
330       9.45          116                  4  5.0  4.5         1   
312       8.78          103                  3  3.5  4.0         0   
333       9.66          117                  4  5.0  4.0         1   

                Chance of Admit  
GRE Score CGPA                   
337       9.65       

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [38]:
print(db.columns)

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA',
       'Research', 'Chance of Admit'],
      dtype='object')


In [39]:
db.set_index(["GRE Score", "CGPA"], inplace = True)


In [40]:
db.reset_index(inplace=True, drop=True)


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [51]:
file_path2 = r'C:\\Users\\David Santos\\Desktop\\Labs IronHack\\Semi-Prontos\\7 - lab-advanced-pandas\\your-code\\data\\Admission_Predict.csv'
db2 = pd.read_csv(file_path2)


db2.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [53]:
filtered_rows = db2[(db2['CGPA'] > 9) & (db2['Research'] == 1)]
num_rows = len(filtered_rows)
print("Number of rows where CGPA > 9 and Research = 1:", num_rows)

Number of rows where CGPA > 9 and Research = 1: 101


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [55]:
filtered_rows2 = db2[(db2['CGPA'] > 9) & (db2['SOP'] < 3.5 )]

mean_chance = filtered_rows2['Chance of Admit '].mean() 

print("Mean 'Chanve of Admit' for selected aapplicants: ", mean_chance)

Mean 'Chanve of Admit' for selected aapplicants:  0.8019999999999999


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [69]:
import numpy as np

def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    mean = np.mean(col)
    std_dev = np.std(col)

    standardized_col = (col-mean) / std_dev

    return standardized_col

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [75]:
db2['CGPA_std'] = standardize(db2['CGPA'])
db2['GRE_std'] = standardize(db2['GRE Score'])
db2['LOR_std'] = standardize(db2['LOR '])

We will generate the decision choice at random using the code below. Please run the cell.

In [76]:
# Libraries
from random import choices

In [77]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=db2.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [81]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=db2.shape[0])


db2['Decision_Choice'] = decision_choice


def lookup_function(value):
    if value > 0.8:
        return "Accepted"
    else: 
        return "Not accepted"
    

db2['Deciding_Column'] = db2.apply(lambda row: lookup_function(row[row['Decision_Choice']]), axis=1)

print(db2)


     Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0             1        337          118                  4  4.5   4.5  9.65   
1             2        316          104                  3  3.0   3.5  8.00   
2             3        322          110                  3  3.5   2.5  8.67   
3             4        314          103                  2  2.0   3.0  8.21   
4             5        330          115                  5  4.5   3.0  9.34   
..          ...        ...          ...                ...  ...   ...   ...   
380         381        324          110                  3  3.5   3.5  9.04   
381         382        325          107                  3  3.0   3.5  9.11   
382         383        330          116                  4  5.0   4.5  9.45   
383         384        312          103                  3  3.5   4.0  8.78   
384         385        333          117                  4  5.0   4.0  9.66   

     Research  Chance of Admit   CGPA_std   GRE_std

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [83]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=db2.shape[0])


db2['Decision_Choice'] = decision_choice


def lookup_function(value):
    if value > 0.8:
        return "Accepted"
    else: 
        return "Not accepted"
    

db2['Deciding_Column'] = db2.apply(lambda row: lookup_function(row[row['Decision_Choice']]), axis=1)

db2['Decision'] = np.where(db2['Deciding_Column'] == 'Accepted', 1, 0)

print(db2)

     Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0             1        337          118                  4  4.5   4.5  9.65   
1             2        316          104                  3  3.0   3.5  8.00   
2             3        322          110                  3  3.5   2.5  8.67   
3             4        314          103                  2  2.0   3.0  8.21   
4             5        330          115                  5  4.5   3.0  9.34   
..          ...        ...          ...                ...  ...   ...   ...   
380         381        324          110                  3  3.5   3.5  9.04   
381         382        325          107                  3  3.0   3.5  9.11   
382         383        330          116                  4  5.0   4.5  9.45   
383         384        312          103                  3  3.5   4.0  8.78   
384         385        333          117                  4  5.0   4.0  9.66   

     Research  Chance of Admit   CGPA_std   GRE_std

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [85]:
num_accepted = db2['Decision'].sum() 

print("Number of applicants accepted: ", num_accepted)

Number of applicants accepted:  89


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [86]:
file_path3 = r'C:\\Users\\David Santos\\Desktop\\Labs IronHack\\Semi-Prontos\\7 - lab-advanced-pandas\\your-code\\data\\Admission_Predict.csv'
db3 = pd.read_csv(file_path2)


db3.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [87]:
db3.columns = db3.columns.str.replace('[^a-zA-Z]', '', regex=True).str.replace(' ', '_').str.lower()

In [88]:
db3.columns

Index(['serialno', 'grescore', 'toeflscore', 'universityrating', 'sop', 'lor',
       'cgpa', 'research', 'chanceofadmit'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [95]:
db3['adjusted_gre'] = db3.apply(
    lambda row: row['grescore'] + 10 if row['universityrating'] >= 4 else row['grescore'],
    axis=1)


bins = [0, 150, 300, 320, db3['adjusted_gre'].max()]
labels = ['Low', 'Moderate', 'High', 'Very High']
db3['adjusted_gre'] = pd.cut(db3['adjusted_gre'], bins=bins, labels=labels)

print(db3)


     serialno  grescore  toeflscore  universityrating  sop  lor  cgpa  \
0           1       337         118                 4  4.5  4.5  9.65   
1           2       316         104                 3  3.0  3.5  8.00   
2           3       322         110                 3  3.5  2.5  8.67   
3           4       314         103                 2  2.0  3.0  8.21   
4           5       330         115                 5  4.5  3.0  9.34   
..        ...       ...         ...               ...  ...  ...   ...   
380       381       324         110                 3  3.5  3.5  9.04   
381       382       325         107                 3  3.0  3.5  9.11   
382       383       330         116                 4  5.0  4.5  9.45   
383       384       312         103                 3  3.5  4.0  8.78   
384       385       333         117                 4  5.0  4.0  9.66   

     research  chanceofadmit adjusted_gre  
0           1           0.92    Very High  
1           1           0.72       